# Document Q&A with Box AI

In this notebook, we will learn how to use the Box AI API `/ask` endpoint to ask questions from a single file in Box. Specifically, we will have access to a PDF file containing federal policy information for the US Parole Commission.

## Prequisites

You must have completed the Setup notebook first. This will create all of the Box objects, folders, and files that you need, and will have created an environment file to help you get started and import all the libraries you will need.

## Workshop

The first step is to import all of the environment variables we need for this exercise.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_FOLDER_ID=os.getenv('EXERCISE2_FOLDER')

Next we will grab the BoxClient object from the Python SDK to authenticate ourselves to the API. We'll print out the current user's information to ensure we are properly authenticated.

In [ ]:
from box_sdk_gen import BoxClient, CCGConfig, BoxCCGAuth

ccg_config = CCGConfig(
    client_id=BOX_CLIENT_ID,
    client_secret=BOX_CLIENT_SECRET,
    user_id=BOX_USER_ID,
)

ccg_auth = BoxCCGAuth(ccg_config)

client = BoxClient(ccg_auth)

print(f"{client.users.get_user_me()}")

Now let's find the file in the exercise folder we'll need for this demo.

In [ ]:
files = client.folders.get_folder_items(BOX_FOLDER_ID)

file_id = ""
for file in files.entries:
    if file.name == "CFR-2024-title28-vol1-sec2-20.pdf":
        file_id = file.id
        break

print(f"File ID: {file_id}")

Now that we have our file ID, we will call the create_ai_ask method with the file ID and our prompt.

In [ ]:
from box_sdk_gen import (
    CreateAiAskMode,
    AiItemAsk,
    AiItemAskTypeField,
    AiDialogueHistory
)

prompt = """
Describe the guidelines used by the US Parole Commission
to determine whether a federal inmate is eligible for parole.
"""

ai_response = client.ai.create_ai_ask(
    CreateAiAskMode.SINGLE_ITEM_QA,
    prompt,
    [
        AiItemAsk(
            id=file_id,
            type=AiItemAskTypeField.FILE,
        )
    ]
)

print (ai_response.answer)

We now have the answer for our question, but what if we want to continue the conversation. REST APIs are stateless, so we provide the dialogue_history object to let Box AI know what we have talked about thus far. 

So let's save the previous question and answer and ask a follow-up question. 

In [ ]:
dialogue_history = []

dialogue_history.append(AiDialogueHistory(
    prompt=prompt,
    answer=ai_response.answer, # type: ignore
    created_at=ai_response.created_at # type: ignore
))

new_response = client.ai.create_ai_ask(
    CreateAiAskMode.SINGLE_ITEM_QA,
    "what can you tell me about the salient factor score",
    [
        AiItemAsk(
            id=file_id,
            type=AiItemAskTypeField.FILE,
        )
    ],
    dialogue_history=dialogue_history,
    include_citations=True
)

print (new_response.answer)
print (f"Citations: {new_response.citations}")


As you can see, the document Q&A capabilities of the Box AI API are pretty powerful. You can ask these questions of a single document or up to 25 document. You can also use the item.content object to inject other data not available in the document itself to enrich the context provided to the LLM for even more tailored answers.

To get you started, we've provided full, runnable files for this exercise. Running the following cells will generate the file for you in the exercise folders. Use it as-is, or use it as inspiration or a starting point for your workflows and applications.

In [ ]:
%%writefile box_ai_qna_single.py
import asyncio
import os

from dotenv import load_dotenv

from box_sdk_gen import (
    AiDialogueHistory,
    AiItemAsk,
    AiItemAskTypeField,
    BoxClient,
    BoxCCGAuth,
    CCGConfig,
    CreateAiAskMode
)

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_FOLDER_ID=os.getenv('EXERCISE2_FOLDER')

def get_file_id(client, folder_id):
    files = client.folders.get_folder_items(folder_id)
    for file in files.entries:
        if file.name == "CFR-2024-title28-vol1-sec2-20.pdf":
            return file.id
    return None

async def chat_with_ai(): 
    ccg_config = CCGConfig(
        client_id=BOX_CLIENT_ID,
        client_secret=BOX_CLIENT_SECRET,
        user_id=BOX_USER_ID,
    )
    auth = BoxCCGAuth(config=ccg_config)
    client = BoxClient(auth=auth)

    box_file_id = get_file_id(client, BOX_FOLDER_ID)    
    
    dialogue_history = []
    
    print("Box AI Chat - Type 'quit' to exit")
    print("-" * 40)
    
    while True:
        prompt = input("\nYou: ").strip()
        
        if prompt.lower() in ['quit', 'exit', 'q']:
            print("Goodbye!")
            break
            
        if not prompt:
            continue
            
        try:
            response = client.ai.create_ai_ask(
                mode=CreateAiAskMode.SINGLE_ITEM_QA,
                prompt=prompt,
                items=[
                    AiItemAsk(
                        id=box_file_id, 
                        type=AiItemAskTypeField.FILE,
                    )
                ],
                dialogue_history=dialogue_history,
                include_citations=True,
            )
            
            print(f"\nAI: {response.answer}") # type: ignore
            
            i=1
            print("\nCitations:")
            print("-" * 20)
            for citation in response.citations:
                print(f"\n{i}. {citation.content} ({citation.name})") # type: ignore
                i += 1
            
            # Update dialogue history
            dialogue_history.append(AiDialogueHistory(
                prompt=prompt,
                answer=response.answer, # type: ignore
                created_at=response.created_at # type: ignore
            ))
            
        except Exception as e:
            print(f"\nError: {e}")

if __name__ == "__main__":
    asyncio.run(chat_with_ai())
